# SQL Básico para Ingeniería de Datos

## Objetivos de Aprendizaje
- Comprender los conceptos fundamentales de bases de datos relacionales
- Dominar las consultas SQL básicas (SELECT, WHERE, JOIN, GROUP BY)
- Trabajar con SQLite desde Python
- Realizar operaciones CRUD (Create, Read, Update, Delete)
- Integrar SQL con Pandas para análisis de datos

## Requisitos
- Python 3.8+
- sqlite3 (incluido en Python)
- pandas
- sqlalchemy

In [ ]:
# Instalación de dependencias
import sys
!{sys.executable} -m pip install pandas sqlalchemy -q

In [ ]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import os

print("SQLite version:", sqlite3.sqlite_version)
print("Pandas version:", pd.__version__)

## 1. Conexión a Base de Datos SQLite

In [ ]:
# Crear conexión a base de datos SQLite (se crea si no existe)
db_path = '../../datasets/raw/tienda.db'
os.makedirs(os.path.dirname(db_path), exist_ok=True)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Conexión establecida a: {db_path}")

## 2. Creación de Tablas

In [ ]:
# Crear tabla de clientes
cursor.execute('''
CREATE TABLE IF NOT EXISTS clientes (
    cliente_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    ciudad TEXT,
    fecha_registro DATE,
    activo INTEGER DEFAULT 1
)
''')

print("Tabla 'clientes' creada")

In [ ]:
# Crear tabla de productos
cursor.execute('''
CREATE TABLE IF NOT EXISTS productos (
    producto_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    categoria TEXT,
    precio REAL NOT NULL,
    stock INTEGER DEFAULT 0,
    fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')

print("Tabla 'productos' creada")

In [ ]:
# Crear tabla de ventas
cursor.execute('''
CREATE TABLE IF NOT EXISTS ventas (
    venta_id INTEGER PRIMARY KEY AUTOINCREMENT,
    cliente_id INTEGER,
    producto_id INTEGER,
    cantidad INTEGER NOT NULL,
    fecha_venta DATE,
    total REAL,
    FOREIGN KEY (cliente_id) REFERENCES clientes(cliente_id),
    FOREIGN KEY (producto_id) REFERENCES productos(producto_id)
)
''')

conn.commit()
print("Tabla 'ventas' creada")

## 3. Inserción de Datos (INSERT)

In [ ]:
# Insertar clientes
clientes_data = [
    ('Juan Pérez', 'juan@email.com', 'Madrid', '2024-01-15'),
    ('María García', 'maria@email.com', 'Barcelona', '2024-02-20'),
    ('Pedro Martínez', 'pedro@email.com', 'Valencia', '2024-03-10'),
    ('Ana López', 'ana@email.com', 'Madrid', '2024-04-05'),
    ('Luis Rodríguez', 'luis@email.com', 'Sevilla', '2024-05-12')
]

cursor.executemany('''
INSERT OR IGNORE INTO clientes (nombre, email, ciudad, fecha_registro)
VALUES (?, ?, ?, ?)
''', clientes_data)

conn.commit()
print(f"Insertados {cursor.rowcount} clientes")

In [ ]:
# Insertar productos
productos_data = [
    ('Laptop Dell XPS', 'Electrónica', 1200.00, 15),
    ('Mouse Logitech', 'Accesorios', 25.00, 50),
    ('Teclado Mecánico', 'Accesorios', 75.00, 30),
    ('Monitor Samsung 27"', 'Electrónica', 300.00, 20),
    ('Webcam HD', 'Accesorios', 80.00, 25),
    ('SSD 1TB', 'Almacenamiento', 150.00, 40),
    ('RAM 16GB', 'Componentes', 100.00, 35)
]

cursor.executemany('''
INSERT OR IGNORE INTO productos (nombre, categoria, precio, stock)
VALUES (?, ?, ?, ?)
''', productos_data)

conn.commit()
print(f"Insertados {len(productos_data)} productos")

In [ ]:
# Insertar ventas
ventas_data = [
    (1, 1, 1, '2024-06-01', 1200.00),
    (2, 2, 2, '2024-06-02', 50.00),
    (3, 3, 3, '2024-06-03', 225.00),
    (1, 4, 1, '2024-06-04', 300.00),
    (4, 5, 2, '2024-06-05', 160.00),
    (2, 6, 1, '2024-06-06', 150.00),
    (5, 2, 5, '2024-06-07', 125.00),
    (3, 1, 2, '2024-06-08', 2400.00),
    (1, 3, 1, '2024-06-09', 75.00),
    (4, 7, 3, '2024-06-10', 300.00)
]

cursor.executemany('''
INSERT OR IGNORE INTO ventas (cliente_id, producto_id, cantidad, fecha_venta, total)
VALUES (?, ?, ?, ?, ?)
''', ventas_data)

conn.commit()
print(f"Insertadas {len(ventas_data)} ventas")

## 4. Consultas Básicas (SELECT)

In [ ]:
# SELECT simple - Todos los clientes
query = "SELECT * FROM clientes"
df_clientes = pd.read_sql_query(query, conn)
print("Todos los clientes:")
print(df_clientes)

In [ ]:
# SELECT con columnas específicas
query = "SELECT nombre, email, ciudad FROM clientes"
df = pd.read_sql_query(query, conn)
print("Clientes - Nombre, Email, Ciudad:")
print(df)

In [ ]:
# SELECT con WHERE - Filtrado
query = "SELECT * FROM productos WHERE categoria = 'Accesorios'"
df = pd.read_sql_query(query, conn)
print("Productos de categoría 'Accesorios':")
print(df)

In [ ]:
# SELECT con múltiples condiciones
query = '''
SELECT nombre, precio, stock
FROM productos
WHERE precio > 100 AND stock > 20
'''
df = pd.read_sql_query(query, conn)
print("Productos con precio > $100 y stock > 20:")
print(df)

In [ ]:
# SELECT con ORDER BY
query = '''
SELECT nombre, precio
FROM productos
ORDER BY precio DESC
LIMIT 5
'''
df = pd.read_sql_query(query, conn)
print("Top 5 productos más caros:")
print(df)

## 5. Funciones de Agregación

In [ ]:
# COUNT, SUM, AVG, MAX, MIN
query = '''
SELECT 
    COUNT(*) as total_productos,
    SUM(stock) as stock_total,
    AVG(precio) as precio_promedio,
    MAX(precio) as precio_maximo,
    MIN(precio) as precio_minimo
FROM productos
'''
df = pd.read_sql_query(query, conn)
print("Estadísticas de productos:")
print(df)

In [ ]:
# GROUP BY - Agrupar por categoría
query = '''
SELECT 
    categoria,
    COUNT(*) as num_productos,
    AVG(precio) as precio_promedio,
    SUM(stock) as stock_total
FROM productos
GROUP BY categoria
ORDER BY num_productos DESC
'''
df = pd.read_sql_query(query, conn)
print("Productos por categoría:")
print(df)

In [ ]:
# GROUP BY con HAVING - Filtrar grupos
query = '''
SELECT 
    categoria,
    COUNT(*) as num_productos,
    AVG(precio) as precio_promedio
FROM productos
GROUP BY categoria
HAVING COUNT(*) >= 2
'''
df = pd.read_sql_query(query, conn)
print("Categorías con 2 o más productos:")
print(df)

## 6. JOINs - Combinar Tablas

In [ ]:
# INNER JOIN - Ventas con información de clientes
query = '''
SELECT 
    v.venta_id,
    c.nombre as cliente,
    v.cantidad,
    v.total,
    v.fecha_venta
FROM ventas v
INNER JOIN clientes c ON v.cliente_id = c.cliente_id
ORDER BY v.fecha_venta
'''
df = pd.read_sql_query(query, conn)
print("Ventas con información de clientes:")
print(df)

In [ ]:
# JOIN múltiple - Ventas completas
query = '''
SELECT 
    v.venta_id,
    c.nombre as cliente,
    p.nombre as producto,
    p.categoria,
    v.cantidad,
    v.total,
    v.fecha_venta
FROM ventas v
INNER JOIN clientes c ON v.cliente_id = c.cliente_id
INNER JOIN productos p ON v.producto_id = p.producto_id
ORDER BY v.fecha_venta DESC
'''
df = pd.read_sql_query(query, conn)
print("Información completa de ventas:")
print(df.head(10))

In [ ]:
# LEFT JOIN - Todos los clientes con sus ventas
query = '''
SELECT 
    c.nombre as cliente,
    COUNT(v.venta_id) as num_compras,
    COALESCE(SUM(v.total), 0) as total_gastado
FROM clientes c
LEFT JOIN ventas v ON c.cliente_id = v.cliente_id
GROUP BY c.cliente_id, c.nombre
ORDER BY total_gastado DESC
'''
df = pd.read_sql_query(query, conn)
print("Resumen de compras por cliente:")
print(df)

## 7. Subconsultas

In [ ]:
# Subconsulta en WHERE
query = '''
SELECT nombre, precio
FROM productos
WHERE precio > (SELECT AVG(precio) FROM productos)
ORDER BY precio DESC
'''
df = pd.read_sql_query(query, conn)
print("Productos con precio mayor al promedio:")
print(df)

In [ ]:
# Subconsulta en FROM
query = '''
SELECT 
    categoria,
    AVG(precio) as precio_promedio
FROM productos
WHERE categoria IN (
    SELECT DISTINCT categoria 
    FROM productos 
    WHERE stock > 20
)
GROUP BY categoria
'''
df = pd.read_sql_query(query, conn)
print("Precio promedio por categoría (con stock > 20):")
print(df)

## 8. Actualización de Datos (UPDATE)

In [ ]:
# UPDATE simple
query = '''
UPDATE productos
SET stock = stock + 10
WHERE categoria = 'Accesorios'
'''
cursor.execute(query)
conn.commit()
print(f"Actualizado stock de {cursor.rowcount} productos")

# Verificar cambios
df = pd.read_sql_query("SELECT nombre, stock FROM productos WHERE categoria = 'Accesorios'", conn)
print("\nStock actualizado:")
print(df)

## 9. Eliminación de Datos (DELETE)

In [ ]:
# Antes de eliminar, ver registros
print("Productos antes de eliminar:")
df = pd.read_sql_query("SELECT COUNT(*) as total FROM productos", conn)
print(df)

# DELETE con condición (comentado para no eliminar datos)
# query = '''
# DELETE FROM productos
# WHERE stock = 0
# '''
# cursor.execute(query)
# conn.commit()
# print(f"Eliminados {cursor.rowcount} productos sin stock")

## 10. Análisis de Datos con SQL y Pandas

In [ ]:
# Análisis 1: Productos más vendidos
query = '''
SELECT 
    p.nombre as producto,
    p.categoria,
    COUNT(v.venta_id) as num_ventas,
    SUM(v.cantidad) as cantidad_total,
    SUM(v.total) as ingresos_totales
FROM productos p
LEFT JOIN ventas v ON p.producto_id = v.producto_id
GROUP BY p.producto_id, p.nombre, p.categoria
HAVING num_ventas > 0
ORDER BY ingresos_totales DESC
'''
df_productos_top = pd.read_sql_query(query, conn)
print("Productos más vendidos:")
print(df_productos_top)

In [ ]:
# Análisis 2: Ventas por ciudad
query = '''
SELECT 
    c.ciudad,
    COUNT(DISTINCT c.cliente_id) as num_clientes,
    COUNT(v.venta_id) as num_ventas,
    SUM(v.total) as ingresos_totales,
    AVG(v.total) as ticket_promedio
FROM clientes c
LEFT JOIN ventas v ON c.cliente_id = v.cliente_id
GROUP BY c.ciudad
ORDER BY ingresos_totales DESC
'''
df_ciudades = pd.read_sql_query(query, conn)
print("Análisis por ciudad:")
print(df_ciudades)

In [ ]:
# Análisis 3: Clientes más valiosos
query = '''
SELECT 
    c.nombre,
    c.email,
    c.ciudad,
    COUNT(v.venta_id) as num_compras,
    SUM(v.total) as total_gastado,
    AVG(v.total) as ticket_promedio,
    MAX(v.fecha_venta) as ultima_compra
FROM clientes c
INNER JOIN ventas v ON c.cliente_id = v.cliente_id
GROUP BY c.cliente_id, c.nombre, c.email, c.ciudad
ORDER BY total_gastado DESC
LIMIT 5
'''
df_top_clientes = pd.read_sql_query(query, conn)
print("Top 5 clientes por valor:")
print(df_top_clientes)

In [ ]:
# Visualización de resultados
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Ingresos por categoría
df_productos_top.groupby('categoria')['ingresos_totales'].sum().plot(
    kind='bar', ax=axes[0], color='skyblue'
)
axes[0].set_title('Ingresos Totales por Categoría')
axes[0].set_ylabel('Ingresos ($)')
axes[0].set_xlabel('Categoría')

# Gráfico 2: Ventas por ciudad
df_ciudades.plot(x='ciudad', y='ingresos_totales', kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Ingresos por Ciudad')
axes[1].set_ylabel('Ingresos ($)')
axes[1].set_xlabel('Ciudad')
axes[1].legend().set_visible(False)

plt.tight_layout()
plt.show()

## 11. Ejercicios Prácticos

In [ ]:
# Ejercicio 1: Encuentra los productos que nunca se han vendido
query = '''
SELECT p.nombre, p.categoria, p.precio, p.stock
FROM productos p
LEFT JOIN ventas v ON p.producto_id = v.producto_id
WHERE v.venta_id IS NULL
'''
df = pd.read_sql_query(query, conn)
print("Productos sin ventas:")
print(df)

In [ ]:
# Ejercicio 2: Calcula el valor total del inventario
query = '''
SELECT 
    categoria,
    SUM(precio * stock) as valor_inventario,
    SUM(stock) as unidades_totales
FROM productos
GROUP BY categoria
ORDER BY valor_inventario DESC
'''
df = pd.read_sql_query(query, conn)
print("Valor del inventario por categoría:")
print(df)
print(f"\nValor total del inventario: ${df['valor_inventario'].sum():,.2f}")

In [ ]:
# Ejercicio 3: Encuentra los días con más ventas
query = '''
SELECT 
    fecha_venta,
    COUNT(*) as num_transacciones,
    SUM(total) as total_dia
FROM ventas
GROUP BY fecha_venta
ORDER BY total_dia DESC
LIMIT 5
'''
df = pd.read_sql_query(query, conn)
print("Top 5 días con más ventas:")
print(df)

## 12. Uso de SQLAlchemy para mayor flexibilidad

In [ ]:
# Crear engine con SQLAlchemy
engine = create_engine(f'sqlite:///{db_path}')

# Leer datos con SQLAlchemy
query = "SELECT * FROM productos"
df_products = pd.read_sql_query(query, engine)
print("Productos (usando SQLAlchemy):")
print(df_products.head())

In [ ]:
# Escribir DataFrame a SQL
df_nuevo = pd.DataFrame({
    'nombre': ['Nuevo Producto 1', 'Nuevo Producto 2'],
    'categoria': ['Test', 'Test'],
    'precio': [99.99, 149.99],
    'stock': [10, 15]
})

# df_nuevo.to_sql('productos', engine, if_exists='append', index=False)
print("DataFrame listo para insertar (comentado para no duplicar datos)")
print(df_nuevo)

## 13. Limpieza

In [ ]:
# Cerrar conexiones
conn.close()
print("Conexión cerrada")

## Resumen y Mejores Prácticas

### Conceptos Clave:
1. **SELECT**: Consultar datos de una o más tablas
2. **WHERE**: Filtrar resultados con condiciones
3. **JOIN**: Combinar datos de múltiples tablas
4. **GROUP BY**: Agrupar datos para agregaciones
5. **ORDER BY**: Ordenar resultados
6. **INSERT/UPDATE/DELETE**: Operaciones CRUD

### Mejores Prácticas SQL:
- Usar nombres descriptivos para tablas y columnas
- Definir claves primarias y foráneas
- Indexar columnas frecuentemente consultadas
- Usar parámetros en lugar de concatenar strings (evitar SQL injection)
- Siempre hacer COMMIT después de modificaciones
- Cerrar conexiones cuando termines
- Usar transacciones para operaciones críticas

### Integración con Pandas:
- `pd.read_sql_query()`: Ejecutar consultas y obtener DataFrame
- `df.to_sql()`: Insertar DataFrame en base de datos
- SQLAlchemy: Mayor flexibilidad y portabilidad

### Recursos Adicionales:
- [SQLite Tutorial](https://www.sqlitetutorial.net/)
- [W3Schools SQL](https://www.w3schools.com/sql/)
- [Pandas SQL](https://pandas.pydata.org/docs/user_guide/io.html#sql-queries)